In [1]:
%%capture
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [2]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import *
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
from model.Embedding import FusionEmbedding,BertEmbedding

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Load Necessary preproceeded Data

In [4]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

In [5]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [6]:
display(len(tr_in))
total_len = len(tr_in)
half = total_len//1000
display(half)
train_split = int(0.80 * half)


770491

770

In [7]:
trainSet = FusionDataset(tr_in[:train_split],tokenizer,\
                         glyph2ix,pinyin2ix,pos2ix,tr_out[:train_split],\
                         tr_pos_in[:train_split],tr_pos_out[:train_split],\
                         device=device) # use device if you want to load it gpu

616it [00:00, 2360.08it/s]
616it [00:00, 2297.67it/s]


In [8]:
valSet = FusionDataset(tr_in[train_split:half],tokenizer,\
                       glyph2ix,pinyin2ix,pos2ix,tr_out[train_split:half],\
                       tr_pos_in[train_split:half],tr_pos_out[train_split:half],\
                       device=device) # use device if you want to load it gpu

154it [00:00, 2402.53it/s]
154it [00:00, 2431.42it/s]


In [9]:
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer, Anchi_Decoder,Anchi_Transformer

In [10]:
from utils.trans_trainer import train

## Fusion_Anchi_Trans_Decoder

In [13]:
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6, # trainable
    'output_dim':21128,# fixed
    'device':device,
}
# batch_size = [32,64,128]
# lr =[0.1,0.01,0.001]
# name = 'anchi_tra_de_128_01_10_'
name = 'test'
train(Fusion_Anchi_Trans_Decoder(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=2,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 24s
	Training Loss: 8.58321 	Validation Loss: 7.02125


 50%|█████     | 1/2 [00:25<00:25, 25.24s/it]

Epoch: 02 | Epoch Time: 0m 23s
	Training Loss: 6.70914 	Validation Loss: 6.72863


100%|██████████| 2/2 [00:49<00:00, 24.66s/it]


In [13]:
model = Fusion_Anchi_Transformer(config).to(device)

In [14]:
res = trainSet[0:2]
bert = Anchibert.to(device)

In [15]:
Xsents_input_ids,Xsents_token_type_ids,\
        Xsents_attention_mask,Xsents_pinyin_ids,\
        Xsents_glyph_ids,Xsents_pos_ids,\
        Ysents_input_ids,Ysents_token_type_ids,\
        Ysents_attention_mask,Ysents_pinyin_ids,\
        Ysents_glyph_ids,Ysents_pos_ids,trueY,y_mask_ids = res

In [16]:
Xword_embeddings = bert(Xsents_input_ids,      \
                         Xsents_token_type_ids, \
                         Xsents_attention_mask  \
                         )['last_hidden_state'].detach()


Yword_embeddings = bert(Ysents_input_ids,      \
                        Ysents_token_type_ids, \
                        Ysents_attention_mask  \
                       )['last_hidden_state'].detach()

inputs = {'Xword_embeddings':Xword_embeddings, \
          'Xsents_pinyin_ids':Xsents_pinyin_ids, \
          'Xsents_glyph_ids':Xsents_glyph_ids,\
          'Xsents_pos_ids':Xsents_pos_ids,\
          'Yword_embeddings':Yword_embeddings,\
          'Ysents_pinyin_ids':Ysents_pinyin_ids, \
          'Ysents_glyph_ids':Ysents_glyph_ids,\
          'Ysents_pos_ids':Ysents_pos_ids, \
          'device':device}
inputs['Xpad_hidden_mask'] = (~ Xsents_attention_mask.bool()).detach()
inputs['Ypad_hidden_mask'] = (~ Ysents_attention_mask.bool()).detach()
inputs['tgt_mask']= y_mask_ids

In [17]:
y_mask_ids.shape

torch.Size([29, 29])

In [18]:
memory = model.encode(**inputs)
model.decode(memory,**inputs).shape

torch.Size([29, 2, 768])

## Anchi_Decoder

In [40]:
config = { # for Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6, # trainable
    'output_dim':21128, # fixed
    'device':device
}
name = 'test'
train(Anchi_Decoder(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=1,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 22s
	Training Loss: 8.37719 	Validation Loss: 6.93155


100%|██████████| 1/1 [00:23<00:00, 23.40s/it]


## Fusion_Anchi_Transformer

In [12]:
config = { # Fusion_Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable 
    'layer_norm_eps':1e-12, 
    'hidden_dropout':0.1, 
    'nhead':12,
    'num_encoder_layers':5, # trainable
    'num_decoder_layers':6, # trainable
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1,
    'device':device
}
# name = 'A_Tran_De_128_01_60_Adam_tag10_e5_d6'
name ='test'
train(Fusion_Anchi_Transformer(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=1,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=5,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 3s
	Training Loss: 10.20252 	Validation Loss: 10.53441


100%|██████████| 1/1 [00:04<00:00,  4.42s/it]


## Anchi_Transformer

In [20]:
config = { # Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'layer_norm_eps':1e-12, 
    'hidden_dropout':0.1, 
    'nhead':12,
    'num_encoder_layers':6, # trainable
    'num_decoder_layers':6, # trainable
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1,
    'device':device
}
name = 'test'
train(Anchi_Transformer(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=5,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 2s
	Training Loss: 10.89027 	Validation Loss: 13.99074


 20%|██        | 1/5 [00:04<00:16,  4.09s/it]

Epoch: 02 | Epoch Time: 0m 2s
	Training Loss: 10.05464 	Validation Loss: 10.73588


 40%|████      | 2/5 [00:08<00:12,  4.11s/it]

...

 60%|██████    | 3/5 [00:12<00:08,  4.14s/it]

Epoch: 04 | Epoch Time: 0m 3s
	Training Loss: 7.54540 	Validation Loss: 9.03403


 80%|████████  | 4/5 [00:16<00:04,  4.12s/it]

Epoch: 05 | Epoch Time: 0m 3s
	Training Loss: 7.17033 	Validation Loss: 8.48137


100%|██████████| 5/5 [00:20<00:00,  4.12s/it]
